In [2]:
import requests
import pandas as pd
import json
import os
from dotenv import load_dotenv
import time # API 서버 부하를 줄이기 위해 time 라이브러리 추가

# --- dotenv 설정 ---
load_dotenv()
api_key = os.getenv("wage_key")
if not api_key:
    print("❌ 오류: .env 파일에서 'wage_key' 키를 찾을 수 없습니다.")
else:
    url = "https://dataapi.wehago.com/data/"
    headers = {
        "apicode": api_key
    }
    
    # --- 반복문 로직 ---
    all_data_list = []  # 모든 데이터를 저장할 빈 리스트 (장바구니)
    start_index = 0
    chunk_size = 500    # 한 번에 요청할 개수 (0~499는 500개)

    print("데이터 수집을 시작합니다...")

    while True:
        # 1. 'from'과 'to' 파라미터를 동적으로 설정
        params = {
            "dataset_key": "DATASET_SUNDO_11",
            "from": start_index,
            "to": start_index + chunk_size - 1
        }
        
        print(f"  -> {start_index}부터 {start_index + chunk_size - 1}까지 요청 중...")

        try:
            response = requests.get(url, headers=headers, params=params)

            if response.status_code == 200:
                data = response.json()
                
                # 2. 'data' 키에서 실제 데이터 리스트를 가져옴
                data_chunk = data.get('data') # .get()을 사용하면 'data' 키가 없어도 오류 안남

                # 3. 데이터가 비어있는지 확인
                if not data_chunk:
                    # 서버가 빈 리스트([])를 반환하면, 더 이상 데이터가 없다는 의미
                    print("✅ 모든 데이터를 수집했습니다. 반복을 종료합니다.")
                    break
                
                # 4. (중요) 가져온 데이터를 '장바구니' 리스트에 추가
                all_data_list.extend(data_chunk)
                
                # 5. 다음 루프를 위해 시작 인덱스 업데이트
                start_index += chunk_size
                
                # (권장) API 서버에 대한 예의로, 요청 사이에 짧은 휴식 추가
                time.sleep(0.1) # 0.1초 대기

            else:
                # API 호출 자체가 실패한 경우 (400, 404, 500 등)
                print(f"❌ 오류 발생 (Status Code: {response.status_code})")
                print(response.text)
                break # 오류 발생 시 반복 중단

        except requests.exceptions.RequestException as e:
            print(f"연결 중 오류가 발생했습니다: {e}")
            break # 오류 발생 시 반복 중단

    # --- 반복문 종료 후 ---
    if all_data_list:
        # 6. '장바구니'에 모아둔 모든 데이터를 하나의 DataFrame으로 변환
        df_total = pd.DataFrame(all_data_list)
        
        print("\n--- 최종 DataFrame 변환 결과 ---")
        print(f"총 {len(df_total)}개의 행(row)을 가져왔습니다.")
        print(df_total.head())
    else:
        print("\n가져온 데이터가 없습니다.")

데이터 수집을 시작합니다...
  -> 0부터 499까지 요청 중...
✅ 모든 데이터를 수집했습니다. 반복을 종료합니다.

가져온 데이터가 없습니다.


In [40]:
df_total.to_csv('명목임금지수.csv')

In [19]:
!pip install dotenv 


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [21]:
import pandas as pd
df=pd.read_csv('명목임금지수.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21457 entries, 0 to 21456
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      21457 non-null  int64  
 1   index           21457 non-null  int64  
 2   UPLFT_XCNTS     21457 non-null  float64
 3   UPLFT_YDNTS     21457 non-null  float64
 4   LOWRGT_XCNTS    21457 non-null  float64
 5   LOWRGT_YDNTS    21457 non-null  float64
 6   ID              21457 non-null  int64  
 7   BSNES_CNT       410 non-null    float64
 8   LABRR_CNT       410 non-null    float64
 9   LABRR_CNT_AVRG  410 non-null    float64
 10  SELNG_AMT       410 non-null    float64
 11  SELNG_AMT_AVRG  410 non-null    float64
 12  SPCE_INFO       21457 non-null  object 
dtypes: float64(9), int64(3), object(1)
memory usage: 2.1+ MB


In [19]:
import pandas as pd

# 1. CSV 불러오기
df1 = pd.read_csv("명목임금지수.csv", encoding='utf-8')

# 2. 컬럼 확인
print(df1.columns.tolist())

# 3. 필요 없는 컬럼 제거 (컬럼명 맞춰서 수정)
df2 = df1.drop(columns=['Unnamed: 0'], errors='ignore')

# 4. 데이터 타입 확인
df2.info()

# 5. NULL 값 제거
df2 = df2.dropna()

df2.head()

['Unnamed: 0', 'index', 'UPLFT_XCNTS', 'UPLFT_YDNTS', 'LOWRGT_XCNTS', 'LOWRGT_YDNTS', 'ID', 'BSNES_CNT', 'LABRR_CNT', 'LABRR_CNT_AVRG', 'SELNG_AMT', 'SELNG_AMT_AVRG', 'SPCE_INFO']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21457 entries, 0 to 21456
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           21457 non-null  int64  
 1   UPLFT_XCNTS     21457 non-null  float64
 2   UPLFT_YDNTS     21457 non-null  float64
 3   LOWRGT_XCNTS    21457 non-null  float64
 4   LOWRGT_YDNTS    21457 non-null  float64
 5   ID              21457 non-null  int64  
 6   BSNES_CNT       410 non-null    float64
 7   LABRR_CNT       410 non-null    float64
 8   LABRR_CNT_AVRG  410 non-null    float64
 9   SELNG_AMT       410 non-null    float64
 10  SELNG_AMT_AVRG  410 non-null    float64
 11  SPCE_INFO       21457 non-null  object 
dtypes: float64(9), int64(2), object(1)
memory usage: 2.0+ MB


,index,UPLFT_XCNTS,UPLFT_YDNTS,LOWRGT_XCNTS,LOWRGT_YDNTS,ID,BSNES_CNT,LABRR_CNT,LABRR_CNT_AVRG,SELNG_AMT,SELNG_AMT_AVRG,SPCE_INFO
71,71,969828.0,2029746.0,970328.0,2029246.0,72,2.0,7.0,4.0,3300.0,1650.0,"POLYGON ((969828.1 2029745.8, 970328.1 2029745..."
72,72,969828.0,2029246.0,970328.0,2028746.0,73,1.0,1.0,1.0,50.0,50.0,"POLYGON ((969828.1 2029245.8, 970328.1 2029245..."
406,406,970328.0,2030246.0,970828.0,2029746.0,407,1.0,1.0,1.0,300.0,300.0,"POLYGON ((970328.1 2030245.8, 970828.1 2030245..."
407,407,970328.0,2029746.0,970828.0,2029246.0,408,10.0,26.0,3.0,4000.0,400.0,"POLYGON ((970328.1 2029745.8, 970828.1 2029745..."
408,408,970328.0,2029246.0,970828.0,2028746.0,409,4.0,8.0,2.0,450.0,113.0,"POLYGON ((970328.1 2029245.8, 970828.1 2029245..."


In [6]:
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv

# 1) .env 파일 불러오기
load_dotenv()
api_key = os.getenv("households_key")   # ✅ 여기! 키 이름 정확하게

if not api_key:
    raise ValueError("❌ .env 파일에서 'households_key' 값을 찾을 수 없습니다.")

# 2) API 기본 정보
url = "https://dataapi.wehago.com/data/"
headers = {"apicode": api_key}
dataset_key = "DATASET_SUNDO_20"   # ✅ 읍면동 전입 가구 수 데이터셋

# 3) 반복 수집 세팅
all_rows = []
start = 0
chunk = 500  # 한 번 요청 당 500개

print("\n🚀 읍면동 전입 가구 수 데이터 다운로드 시작\n")

while True:
    params = {
        "dataset_key": dataset_key,
        "from": start,
        "to": start + chunk - 1
    }

    print(f"👉 요청: {start} ~ {start + chunk - 1}")

    response = requests.get(url, headers=headers, params=params)

    if response.status_code != 200:
        print(f"❌ 오류 발생 (코드 {response.status_code})\n{response.text}")
        break

    data_chunk = response.json().get("data", [])

    if not data_chunk:  # 더 이상 데이터 없으면 종료
        print("\n✅ 모든 데이터를 성공적으로 수집했습니다!")
        break

    all_rows.extend(data_chunk)
    start += chunk
    time.sleep(0.1)  # 서버 과부하 방지

# 4) DataFrame 변환 + CSV 저장
if all_rows:
    df = pd.DataFrame(all_rows)
    print(f"\n📌 총 {len(df)}개의 행을 수집했습니다.")
    
    save_name = "읍면동_전입가구수.csv"
    df.to_csv(save_name, index=False, encoding="utf-8-sig")

    print(f"💾 CSV 저장 완료 → {save_name}\n")
else:
    print("\n⚠️ 가져온 데이터가 없습니다.\n")


🚀 읍면동 전입 가구 수 데이터 다운로드 시작

👉 요청: 0 ~ 499
👉 요청: 500 ~ 999
👉 요청: 1000 ~ 1499
👉 요청: 1500 ~ 1999
👉 요청: 2000 ~ 2499
👉 요청: 2500 ~ 2999
👉 요청: 3000 ~ 3499
👉 요청: 3500 ~ 3999
👉 요청: 4000 ~ 4499
👉 요청: 4500 ~ 4999
👉 요청: 5000 ~ 5499
👉 요청: 5500 ~ 5999
👉 요청: 6000 ~ 6499
👉 요청: 6500 ~ 6999
👉 요청: 7000 ~ 7499
👉 요청: 7500 ~ 7999
👉 요청: 8000 ~ 8499
👉 요청: 8500 ~ 8999
👉 요청: 9000 ~ 9499
👉 요청: 9500 ~ 9999
👉 요청: 10000 ~ 10499
👉 요청: 10500 ~ 10999
👉 요청: 11000 ~ 11499
👉 요청: 11500 ~ 11999
👉 요청: 12000 ~ 12499
👉 요청: 12500 ~ 12999
👉 요청: 13000 ~ 13499
👉 요청: 13500 ~ 13999
👉 요청: 14000 ~ 14499
👉 요청: 14500 ~ 14999

✅ 모든 데이터를 성공적으로 수집했습니다!

📌 총 14178개의 행을 수집했습니다.
💾 CSV 저장 완료 → 읍면동_전입가구수.csv



In [10]:
df.to_csv("읍면동_전입가구수.csv", index=False, encoding="utf-8-sig")

In [8]:
!pip install dotenv 


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import pandas as pd
df1 = pd.read_csv("읍면동_전입가구수.csv", encoding='utf-8')
#어떤 columns 있는지 확인 
df1.head()
#데이터프레임의 상위 5행을 출력
df1.columns.tolist()
#데이터프레임의 전체 구조 요약 정보 출력
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14178 entries, 0 to 14177
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 14178 non-null  int64  
 1   TRNSFRN_AREA          14178 non-null  object 
 2   AVRG_AGE              14172 non-null  float64
 3   SEXDSTN               14178 non-null  object 
 4   TRNSFRN_HSHLD_CNT_SM  14172 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 554.0+ KB
